In [1]:
import pandas as pd

# handle fasta files
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import seaborn as sns
import glob
from collections import Counter

In [2]:
##change to path of condor-analysis
path_local = "U:/users/mamorel/Projet_Convergence/condor-analysis/"




In [3]:
#workdir = "JTT_freshbrackish"
#workdir = "JTT_marine"
workdir = "LG_freshbrackish"
#workdir = "LG_marine"

for workdir in ["LG_freshbrackish", "LG_marine"]:
    all_substitutions = pd.read_csv(
        path_local
        + "results/rhodopsin/condor/"
        + workdir
        + "/tested_results.tsv",
        sep="\t",
    )
    risk = 0.1

    interest = all_substitutions[all_substitutions.adjust_pvalue <= risk]
    pos_mut = [
        "".join([str(i + 12), j]) for i, j in zip(interest.position, interest.mut)
    ]
    print(workdir, len(interest), set(pos_mut).intersection(["83D", "261Y", "292S", "83N", "122Q", "122E", "292A", "261F"]))

LG_freshbrackish 60 {'292S', '83D', '261Y', '83N'}
LG_marine 60 {'292S', '83D', '261Y', '83N'}


## Number of detections

In [8]:
#"LG_freshbrackish", "LG_marine", "JTT_freshbrackish", "JTT_marine"


#1st with freshbrackish environment

#workdir = "LG_freshbrackish" 
#pcoc_results = pd.read_csv(path_local+"results/rhodopsin/PCOC/pcoc_pastml_freshbrackish/lessgappy.align_reroot.results.tsv", sep="\t")

#2nd with marine environment
workdir = "LG_marine"
pcoc_results = pd.read_csv(path_local+"results/rhodopsin/PCOC/pcoc_pastml_marine/lessgappy.align_reroot.results.tsv", sep="\t")

all_substitutions = pd.read_csv(
    path_local
    + "results/rhodopsin/condor/"
    + workdir
    + "/tested_results.tsv",
    sep="\t",
)



subset_pcoc_results = pd.DataFrame(pcoc_results[pcoc_results.Sites.isin(all_substitutions.position)])
subset_pcoc_results.rename(columns={"Sites": "position"}, inplace = True)
All_bayes_pcoc = pd.merge(all_substitutions, subset_pcoc_results, on="position")

FADE_file = path_local+"results/rhodopsin/FADE/datamonkey-rhodo_conjunction_"+workdir+".csv"

FADE = pd.read_csv(FADE_file, sep=",")
FADE.columns = ["mut", "position", "rate", "bias", "proba", "BF_fade", "composition", "substitutions"]
FADE["posmut"] = FADE["position"].astype(str)+FADE["mut"]
FADE_tested = pd.DataFrame(FADE[FADE.posmut.isin(all_substitutions["posmut"])])

All_bayes_pcoc_fade = pd.merge(All_bayes_pcoc, FADE_tested , on=["posmut", "position", "mut"])

for BF in [100, 1000, 2000]:
    FADE = All_bayes_pcoc_fade[All_bayes_pcoc_fade.BF_fade > BF]
    pos_mut = [
        "".join([str(i + 12), j]) for i, j in zip(FADE.position, FADE.mut)
    ]
    print(workdir,"FADE",BF, len(FADE), set(pos_mut).intersection(["83D", "261Y", "292S", "83N", "122Q", "122E", "292A", "261F"]))
    #print(len(interest)) 


pval = 0.1
Emergence = all_substitutions[ all_substitutions.adjust_pvalue <= pval]
pos_mut = [
    "".join([str(i + 12), j]) for i, j in zip(Emergence.position, Emergence.mut)
]
print(workdir,"Emergence",pval, len(Emergence), set(pos_mut).intersection(["83D", "261Y", "292S", "83N", "122Q", "122E", "292A", "261F"]))
    # #print(len(interest))

BF = 20
Correlation = all_substitutions[(all_substitutions.BF >= BF) & (all_substitutions.correlation == "positive")]
pos_mut = [
    "".join([str(i + 12), j]) for i, j in zip(Correlation.position, Correlation.mut)
]
print(workdir,"correlation", BF, len(Correlation), set(pos_mut).intersection(["83D", "261Y", "292S", "83N", "122Q", "122E", "292A", "261F"]))
    # #print(len(interest))
Condor = all_substitutions[(all_substitutions.BF >= BF) & (all_substitutions.correlation == "positive") & (all_substitutions.adjust_pvalue <= pval)]
pos_mut = [
    "".join([str(i + 12), j]) for i, j in zip(Condor.position, Condor.mut)
]
print(workdir,"Condor",BF, len(Condor), set(pos_mut).intersection(["83D", "261Y", "292S", "83N", "122Q", "122E", "292A", "261F"]))
    # #print(len(interest))     
    
pc = 0.8
interest = All_bayes_pcoc[(All_bayes_pcoc.PC >= pc) & (All_bayes_pcoc.correlation == "positive")]
pos_mut = [
    "".join([str(i + 12), j]) for i, j in zip(interest.position, interest.mut)
]
print(workdir,"pcoc",pc, len(interest), set(pos_mut).intersection(["83D", "261Y", "292S", "83N", "122Q", "122E", "292A", "261F"]))


LG_marine FADE 100 55 {'292S', '83N'}
LG_marine FADE 1000 40 {'292S', '83N'}
LG_marine FADE 2000 36 {'292S'}
LG_marine Emergence 0.1 60 {'292S', '83D', '261Y', '83N'}
LG_marine correlation 20 33 {'292S', '261F', '83N'}
LG_marine Condor 20 9 {'292S', '83N'}
LG_marine pcoc 0.8 9 {'122Q'}


## Rankings 

In [25]:
#"LG_freshbrackish", "LG_marine", "JTT_freshbrackish", "JTT_marine"

#1st with freshbrackish environment
workdir = "LG_freshbrackish" 
pcoc_results = pd.read_csv(path_local+"results/rhodopsin/PCOC/pcoc_pastml_freshbrackish/lessgappy.align_reroot.results.tsv", sep="\t")

#2nd with marine environment
#workdir = "LG_marine"
#pcoc_results = pd.read_csv(path_local+"results/rhodopsin/PCOC/pcoc_pastml_marine/lessgappy.align_reroot.results.tsv", sep="\t")

all_substitutions = pd.read_csv(
    path_local
    + "results/rhodopsin/condor/"
    + workdir
    + "/tested_results.tsv",
    sep="\t",
)


subset_pcoc_results = pd.DataFrame(pcoc_results[pcoc_results.Sites.isin(all_substitutions.position)])
subset_pcoc_results.rename(columns={"Sites": "position"}, inplace = True)
All_bayes_pcoc = pd.merge(all_substitutions, subset_pcoc_results, on="position")


FADE_file = path_local+"results/rhodopsin/FADE/datamonkey-rhodo_conjunction_"+workdir+".csv"

FADE = pd.read_csv(FADE_file, sep=",")
FADE.columns = ["mut", "position", "rate", "bias", "proba", "BF_fade", "composition", "substitutions"]
FADE["posmut"] = FADE["position"].astype(str)+FADE["mut"]
FADE_tested = pd.DataFrame(FADE[FADE.posmut.isin(all_substitutions["posmut"])])

All_bayes_pcoc_fade = pd.merge(All_bayes_pcoc, FADE_tested , on=["posmut", "position", "mut"])

pos_corr = All_bayes_pcoc_fade[All_bayes_pcoc_fade.correlation == "positive"]

method = "ConDor" #FADE #PCOC, Emergence, Correlation, ConDor

if "freshbrackish" in workdir : 
    for mut_i in ["83D", "122E", "261Y", "292A", "166S"]:
        if method == "FADE":
            DF_sort = All_bayes_pcoc_fade.sort_values(by = ['BF_fade'], ascending=[False])
        elif method == "Correlation":
            DF_sort = All_bayes_pcoc_fade[(All_bayes_pcoc_fade.correlation == "positive")].sort_values(by = ['BF'], ascending=[False])
        elif method == "PCOC":
            DF_sort = All_bayes_pcoc_fade[(All_bayes_pcoc_fade.correlation == "positive")].sort_values(by = ['PC'], ascending=[False])
        elif method == "Emergence":
            DF_sort = All_bayes_pcoc_fade[(All_bayes_pcoc_fade.correlation == "positive")].sort_values(by = ['pvalue_raw'], ascending=[True])
        else:
            DF_sort = All_bayes_pcoc_fade[(All_bayes_pcoc_fade.correlation == "positive") & (All_bayes_pcoc_fade.pvalue_raw < 0.1 )].sort_values(by = ['pvalue_raw', 'BF'], ascending=[True,False])
        pos_mut = [
            "".join([str(i + 12), j]) for i, j in zip(DF_sort.position, DF_sort.mut)
        ]
        try:
            print(workdir, method, mut_i, pos_mut.index(mut_i)+1)
        except:
            pass  
else:
    for mut_i in ["83N", "122Q", "261F", "292S"]:
        if method == "FADE":
            DF_sort = All_bayes_pcoc_fade.sort_values(by = ['BF_fade'], ascending=[False])
        elif method == "Correlation":
            DF_sort = All_bayes_pcoc_fade[(All_bayes_pcoc_fade.correlation == "positive")].sort_values(by = ['BF'], ascending=[False])
        elif method == "PCOC":
            DF_sort = All_bayes_pcoc_fade[(All_bayes_pcoc_fade.correlation == "positive")].sort_values(by = ['PC'], ascending=[False])
        elif method == "Emergence":
            DF_sort = All_bayes_pcoc_fade[(All_bayes_pcoc_fade.correlation == "positive")].sort_values(by = ['pvalue_raw'], ascending=[True])
        else:
            DF_sort = All_bayes_pcoc_fade[(All_bayes_pcoc_fade.correlation == "positive") & (All_bayes_pcoc_fade.pvalue_raw < 0.1 )].sort_values(by = ['pvalue_raw', 'BF'], ascending=[True,False])
        pos_mut = [
            "".join([str(i + 12), j]) for i, j in zip(DF_sort.position, DF_sort.mut)
        ]
        try:
            print(workdir, method, mut_i, pos_mut.index(mut_i)+1)
        except:
            pass 
    

LG_freshbrackish ConDor 83D 1
LG_freshbrackish ConDor 122E 94
LG_freshbrackish ConDor 261Y 5
LG_freshbrackish ConDor 166S 3
